# Exploring Functionality (Folktables)

This notebook shows how each argument changes results and runtime using ACS (Folktables):

- input modes (DataFrame / CSV / two-sample)

- protected_list, continuous_list, fp_map

- n_samples, seed

- method="MSD" vs method="l_inf"

## Imports & basics

In [1]:
import numpy as np
import pandas as pd

from time import perf_counter

from folktables import ACSIncome

from humancompatible.detect import (
    detect_and_score,
    most_biased_subgroup, evaluate_biased_subgroup,
    most_biased_subgroup_csv, evaluate_biased_subgroup_csv,
    most_biased_subgroup_two_samples, evaluate_biased_subgroup_two_samples,
)
from humancompatible.detect.helpers import report_subgroup_bias

from supports.folktables_utils import (
    load_state_data,
    ProtectedOnly,
    FEATURE_PROCESSING,
    CONTINUOUS_FEATURES,
    FEATURE_NAMES,
    PROTECTED_VALUES_MAP,
)

STATE_A = "CA"
STATE_B = "FL"
STATE_C = "NH"

## Quick API recap

**Core helper**  
`detect_and_score(...) -> (rule, value)` works in 3 modes:
- DataFrame: pass X, y
- CSV: pass csv_path, target_col
- Two-sample: pass X1, X2

Other shared options:
- protected_list, continuous_list, fp_map
- n_samples, seed
- method="MSD" (default) or "l_inf" (+ method_kwargs)